In [13]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from scipy.stats import norm
import time as ttt
import iisignature as iisig
from tqdm import *
from einops import rearrange

In [14]:
import torch
import torch.nn as nn
import pandas as pd 

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

data_type=torch.float32

In [15]:
x0 = 100.0 # initial condition
sigma = 0.15 # volatility
d=1
K=100.0

segs=20
r = 0.05 # risk free rate
batch_size = 100 # batch size
steps=1000
T = 1 # maturity
dt = T/steps # mesh size

sqrt_dt=np.sqrt(dt); 
dt_new = T/segs # new mesh after shrinkage
level = 3 # truncation level logsig 91 sig 258

In [16]:
batch_in=100
MOMENTUM = 0.99
EPSILON = 1e-6
import warnings
warnings.filterwarnings("ignore")

In [17]:
def create_stock(x0,r,sigma,T,steps,n_path,dW):  
    s_vec=[];
    w0=np.ones((n_path,1,d))*1e-6
    dW=np.concatenate((w0,dW),axis=1) ## the first time slot is then 0
    s_vec.append(np.ones((n_path,d))*100.0)
    for i in range(steps): 
        s_vectemp=s_vec[-1]+ r*s_vec[-1]*dt+ dW[:,i+1,:]*s_vec[-1]*sigma
        s_vec.append(s_vectemp)
    BM_path=np.cumsum(dW,axis=1) ## find the cumulative sum
    S_path=rearrange(np.array(s_vec), 'b c h -> c b h') 
    return BM_path, S_path

def jointime(T,path): 
    n_path, steps, d=path.shape
    dt=T/(steps-1); 
    
    times=np.arange(0,T,dt) ## This can be taken out
    times=np.append(times,T); 
    times_vec=np.tile(times,[1,1]); 
    times_vec=np.transpose(times_vec)
    times_vec=np.tile(times_vec,[n_path,1,1])
    times_vec=np.concatenate((times_vec, path),axis=2)
    return times_vec

def ComputeMultiLevelSig(path, number_of_segment, depth,log_sig=False):
    n_batch, nsteps, chanels = path.shape
    t_vec = np.arange(0, nsteps-1, int(nsteps / number_of_segment))
    t_vec = np.append(t_vec, nsteps-1)
    MultiLevelSig = []
    s=iisig.prepare(d+1,depth)
    
    if log_sig: 
        ll=iisig.logsig(np.expand_dims(path[:,0,:],axis=1),s)
        MultiLevelSig.append(ll)
        for i in range(len(t_vec)-1):    
        ## Notice that we only use the signature of the concatenation of time and space.
            MultiLevelSig.append(iisig.logsig(path[:,0:t_vec[i+1]+1,:],s)) ##if not
        MultiLevelSig=np.stack(MultiLevelSig)  
        MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
    else: 
        ll=iisig.sig(np.expand_dims(path[:,0,:],axis=1),depth)
        MultiLevelSig.append(ll)
        for i in range(len(t_vec)-1):    
        ## Notice that we only use the signature of the concatenation of time and space.
            MultiLevelSig.append(iisig.sig(path[:,0:t_vec[i+1]+1,:],depth)) ##if not
            #MultiLevelSig.append(path_class.signature(t_vec[i],t_vec[i+1]+1))
        MultiLevelSig=np.stack(MultiLevelSig)  
        MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
        
    return MultiLevelSig

In [18]:
dW=np.sqrt(dt)*np.random.normal(size=(500, steps,d))
bm_p,s_path=create_stock(x0,r,sigma,T,steps,500,dW)
tv=jointime(T,bm_p)
ms=ComputeMultiLevelSig(tv, segs, 3,log_sig=False)

In [19]:
ms.shape

(500, 21, 14)

In [26]:
### The x_ten here has to be a path
### The level of discretization is on the fine mesh
def terminal_loss(x_ten):
    res=torch.relu(torch.sum(x_ten[:,:-1,:],dim=(1,2))*dt/d-K)
    return res

In [27]:
def generate_samples(batch_in=100):
    """
    Produce the signature, dW for computation, YT the terminal condition
    x_ten, selection the path and location to be used
    """
    dW=np.sqrt(dt)*np.random.normal(size=(batch_in, steps,d))
    
    pth2=create_stock(x0,r,sigma,T,steps,batch_in,dW)
    BM_timePath=jointime(T,pth2[0]); 
    S_timePath=jointime(T,pth2[1]);
    
    sigs=ComputeMultiLevelSig(S_timePath, 20, 3)
    selection = np.linspace(0,steps, segs+1, dtype = np.int)

    BM_seg=BM_timePath[:,selection,1:]
    dW=BM_seg[:,1:,:]-BM_seg[:,:-1,:]

    dW=torch.tensor(dW,dtype=torch.float32)
    sigs=torch.tensor(sigs,dtype=torch.float32)
    S_timePath=torch.tensor(S_timePath,dtype=torch.float32)
    
    x_ten=S_timePath[:,:,1:]
    
    YT=torch.relu(torch.sum(x_ten[:,:-1,:],dim=(1,2))*dt/d-K)
    
    return sigs, dW, YT, x_ten, selection[:-1]

In [28]:
class Config(object):
    n_layer = 4
    batch_size = 1024
    valid_size = 1024
    
    dim=14; 
    Ntime=20; 
    delta=1/Ntime
    sqrt_deltaT=np.sqrt(1.0/Ntime); 
    lam=1; 

    logging_frequency = 100
    verbose = True
    y_init_range = [0, 1]
    
    num_hiddens = [dim,64,64,d] ## 256 ,256
    
def get_config(name):
    try:
        return globals()[name]
    except KeyError:
        raise KeyError("config not defined.")

cfg=get_config('Config')

In [29]:
class Dense(nn.Module): 
    def __init__(self,cin, cout, batch_norm=False, activate=True): 
        super(Dense,self).__init__()
        self.cin=cin; 
        self.cout=cout; 
        self.activate=activate; 
        
        self.linear=nn.Linear(self.cin,self.cout) #The linear layer
        #BatchNorm1d: it requires the input to be a correct size
        if batch_norm: 
            self.bn=nn.BatchNorm1d(cout,eps=EPSILON,momentum=MOMENTUM)
        else: 
            self.bn=None
      #  nn.init.normal_(self.linear.weight,std=5.0/np.sqrt(cin+cout))
        # This is the He initialization
        
    def forward(self,x): 
        x=self.linear(x)
        if self.bn is not None:
            x=self.bn(x)
        if self.activate:
            x=torch.relu(x)
        return x 
    
class FFN(nn.Module):
    def __init__(self, config):
        super(FFN,self).__init__()
        self.config=config
        
        self.bn=nn.BatchNorm1d(config.num_hiddens[0],eps=EPSILON,momentum=MOMENTUM) ## So there is batch norm no problem
        # range(1,5): 1,2,3,4
        self.layers=[Dense(config.num_hiddens[i-1],config.num_hiddens[i]) for i in range(1, len(config.num_hiddens)-1)]
        self.layers+=[Dense(config.num_hiddens[-2], config.num_hiddens[-1],activate=False)]
        self.layers=nn.Sequential(*self.layers)
    
    def forward(self,x):
      #  x=self.bn(x)
        x=self.layers(x)
        return x 
    
class Lookback_PPDE_Backward(nn.Module):
    def __init__(self,cfg): 
        super(Lookback_PPDE_Backward,self).__init__()
        self.cfg=cfg
        self.Ntime=self.cfg.Ntime 
        self.mList=nn.ModuleList([FFN(self.cfg) for _ in range(self.Ntime)])
        
    def forward(self,batch_sig,batch_dW,batch_YT, batch_x, batch_sel): 
        Y=batch_YT
        for i in np.arange(segs-1,-1, -1):
            Y=Y-Y*r*T/segs-sigma*torch.sum( self.mList[i](batch_sig[:,i,:])*batch_dW[:,i,:],axis=1,keepdim=True)
            gi=terminal_loss(batch_x[:,:batch_sel[i],:])
            Y=torch.relu(gi-Y)+Y
        return Y

In [30]:
def loss_var(x):
    temp=torch.var(x)
    return temp

In [31]:
import torch.optim as optim
from torch.nn import Parameter
import math
model_PPDE_bw=Lookback_PPDE_Backward(cfg)
model_PPDE_bw
optimizer=optim.Adam(model_PPDE_bw.parameters(),lr=5e-4)
grad_clip=0.2
y0_mean=[];
loss_vec=[];
#scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=80, gamma=0.1)

## 0.5823 -- 0.5784 -- 0.5862

In [33]:
for i in range(100):
    batch_sig, batch_dw, batch_y,batch_x, batch_sel =generate_samples(batch_in=1000)

    x_temp=model_PPDE_bw(batch_sig,batch_dw,batch_y,batch_x, batch_sel)
    loss_temp=loss_var(x_temp)

    optimizer.zero_grad()
    loss_temp.backward()
    optimizer.step()
    y0_val=x_temp.mean().cpu().detach().numpy()
    loss_val=loss_temp.cpu().detach().numpy()
    
    y0_mean.append(y0_val)
    loss_vec.append(loss_val)
    if grad_clip: 
        nn.utils.clip_grad_value_(model_PPDE_bw.parameters(), grad_clip)
    
    print("Iter:", i, 'The mean Y0 is', y0_val , 'Variance is:' ,loss_val)

Iter: 0 The mean Y0 is 5.8460164 Variance is: 63.677483
Iter: 1 The mean Y0 is 5.613887 Variance is: 48.51224
Iter: 2 The mean Y0 is 5.604537 Variance is: 43.026905
Iter: 3 The mean Y0 is 5.2322025 Variance is: 41.268795
Iter: 4 The mean Y0 is 5.4867 Variance is: 43.391792
Iter: 5 The mean Y0 is 5.161332 Variance is: 41.43737
Iter: 6 The mean Y0 is 5.324105 Variance is: 45.53004
Iter: 7 The mean Y0 is 5.4254336 Variance is: 43.26021
Iter: 8 The mean Y0 is 4.8258295 Variance is: 39.792824
Iter: 9 The mean Y0 is 4.6002846 Variance is: 30.146809
Iter: 10 The mean Y0 is 4.6761665 Variance is: 33.980087
Iter: 11 The mean Y0 is 5.118485 Variance is: 37.866997
Iter: 12 The mean Y0 is 5.384235 Variance is: 39.366745
Iter: 13 The mean Y0 is 5.0874734 Variance is: 36.11031
Iter: 14 The mean Y0 is 5.084626 Variance is: 36.711796
Iter: 15 The mean Y0 is 5.201277 Variance is: 41.15076
Iter: 16 The mean Y0 is 5.1670136 Variance is: 38.58616
Iter: 17 The mean Y0 is 5.293633 Variance is: 38.796772
Ite

In [34]:
df=pd.DataFrame()
df['y_pred']=y0_mean
df['loss_var']=loss_vec

In [35]:
#5.037, 4.93, 4.915, 4.93

In [39]:
df_temp=df[41:]
lw=df_temp.loss_var.mean()-2.0*df_temp.loss_var.std()
up=df_temp.loss_var.mean()+2.0*df_temp.loss_var.std()
df1=df_temp[df_temp.loss_var>=lw]
df1=df1[df1.loss_var<=up]
df1.y_pred.mean()

5.095399584089007

In [43]:
for epoch in np.arange(1,51,1):
    model_PPDE_bw=Lookback_PPDE_Backward(cfg)
    optimizer=optim.Adam(model_PPDE_bw.parameters(),lr=5e-4)
    grad_clip=0.2
    y0_mean=[];
    loss_vec=[];
    for i in range(100):
        batch_sig, batch_dw, batch_y,batch_x, batch_sel =generate_samples(batch_in=1000)

        x_temp=model_PPDE_bw(batch_sig,batch_dw,batch_y,batch_x, batch_sel)
        loss_temp=loss_var(x_temp)

        optimizer.zero_grad()
        loss_temp.backward()
        optimizer.step()
    #   scheduler.step()
        y0_val=x_temp.mean().cpu().detach().numpy()
        loss_val=loss_temp.cpu().detach().numpy()

        y0_mean.append(y0_val)
        loss_vec.append(loss_val)
        if grad_clip: 
            nn.utils.clip_grad_value_(model_PPDE_bw.parameters(), grad_clip)

        print("Iter:", i, 'The mean Y0 is', y0_val , 'Variance is:' ,loss_val)
    
    print("Epoch:", epoch)
    
    y_pred=np.array(y0_mean)
    loss_val=np.array(loss_vec)
    iters=np.arange(0,len(y_pred),1)+1
    df=pd.DataFrame()
    df['iter']=iters
    df['y_pred']=y_pred
    df['loss_var']=loss_val
    df.to_csv('eg2_trained_dataDim1/method2'+'_'+str(epoch)+'.csv')

Iter: 0 The mean Y0 is 5.806349 Variance is: 55.924618
Iter: 1 The mean Y0 is 5.037728 Variance is: 35.571144
Iter: 2 The mean Y0 is 5.314929 Variance is: 41.29559
Iter: 3 The mean Y0 is 5.3258023 Variance is: 37.442505
Iter: 4 The mean Y0 is 4.9152274 Variance is: 34.416187
Iter: 5 The mean Y0 is 4.6076837 Variance is: 32.68019
Iter: 6 The mean Y0 is 5.12671 Variance is: 39.607243
Iter: 7 The mean Y0 is 5.265826 Variance is: 36.80348
Iter: 8 The mean Y0 is 4.7129154 Variance is: 35.626587
Iter: 9 The mean Y0 is 5.197254 Variance is: 35.626225
Iter: 10 The mean Y0 is 5.302865 Variance is: 37.082214
Iter: 11 The mean Y0 is 5.0353994 Variance is: 35.78975
Iter: 12 The mean Y0 is 4.937721 Variance is: 33.267387
Iter: 13 The mean Y0 is 4.809776 Variance is: 33.961094
Iter: 14 The mean Y0 is 5.065603 Variance is: 34.121597
Iter: 15 The mean Y0 is 5.378037 Variance is: 39.05891
Iter: 16 The mean Y0 is 5.0647173 Variance is: 37.1675
Iter: 17 The mean Y0 is 5.3094945 Variance is: 37.28996
Iter

Iter: 47 The mean Y0 is 4.9646225 Variance is: 35.026604
Iter: 48 The mean Y0 is 4.86621 Variance is: 30.312359
Iter: 49 The mean Y0 is 5.0431156 Variance is: 34.817444
Iter: 50 The mean Y0 is 4.951564 Variance is: 35.109802
Iter: 51 The mean Y0 is 5.246656 Variance is: 35.082855
Iter: 52 The mean Y0 is 5.0526056 Variance is: 34.480026
Iter: 53 The mean Y0 is 4.8144746 Variance is: 33.38313
Iter: 54 The mean Y0 is 4.9416995 Variance is: 33.649532
Iter: 55 The mean Y0 is 5.1329374 Variance is: 35.8686
Iter: 56 The mean Y0 is 4.5364437 Variance is: 29.865978
Iter: 57 The mean Y0 is 4.7392006 Variance is: 32.265053
Iter: 58 The mean Y0 is 5.1478944 Variance is: 36.101032
Iter: 59 The mean Y0 is 5.037792 Variance is: 35.557823
Iter: 60 The mean Y0 is 4.6410546 Variance is: 31.738537
Iter: 61 The mean Y0 is 5.061622 Variance is: 35.913666
Iter: 62 The mean Y0 is 5.1206055 Variance is: 34.536083
Iter: 63 The mean Y0 is 4.9807677 Variance is: 37.281673
Iter: 64 The mean Y0 is 5.202358 Varianc

Iter: 94 The mean Y0 is 4.988269 Variance is: 34.230408
Iter: 95 The mean Y0 is 4.9795203 Variance is: 35.10075
Iter: 96 The mean Y0 is 4.9268637 Variance is: 34.40959
Iter: 97 The mean Y0 is 4.916651 Variance is: 32.879597
Iter: 98 The mean Y0 is 5.422164 Variance is: 38.472427
Iter: 99 The mean Y0 is 5.1706367 Variance is: 36.830532
Epoch: 3
Iter: 0 The mean Y0 is 5.1468897 Variance is: 37.753498
Iter: 1 The mean Y0 is 5.4801164 Variance is: 43.500744
Iter: 2 The mean Y0 is 5.1981153 Variance is: 44.063126
Iter: 3 The mean Y0 is 5.3181643 Variance is: 43.556217
Iter: 4 The mean Y0 is 5.2217593 Variance is: 42.13512
Iter: 5 The mean Y0 is 5.1913204 Variance is: 34.614235
Iter: 6 The mean Y0 is 5.315874 Variance is: 38.3002
Iter: 7 The mean Y0 is 5.012854 Variance is: 36.40246
Iter: 8 The mean Y0 is 4.56764 Variance is: 31.724003
Iter: 9 The mean Y0 is 4.910252 Variance is: 34.836273
Iter: 10 The mean Y0 is 5.055219 Variance is: 36.68583
Iter: 11 The mean Y0 is 5.2023907 Variance is: 3

Iter: 41 The mean Y0 is 4.9356976 Variance is: 33.87175
Iter: 42 The mean Y0 is 4.928177 Variance is: 34.96718
Iter: 43 The mean Y0 is 5.2205734 Variance is: 36.43822
Iter: 44 The mean Y0 is 5.075812 Variance is: 35.541367
Iter: 45 The mean Y0 is 4.930062 Variance is: 33.855827
Iter: 46 The mean Y0 is 5.2528043 Variance is: 37.541817
Iter: 47 The mean Y0 is 5.2064605 Variance is: 35.35221
Iter: 48 The mean Y0 is 5.473229 Variance is: 39.953278
Iter: 49 The mean Y0 is 5.0798955 Variance is: 36.375576
Iter: 50 The mean Y0 is 5.1669335 Variance is: 36.400673
Iter: 51 The mean Y0 is 5.0102377 Variance is: 35.294083
Iter: 52 The mean Y0 is 5.141347 Variance is: 37.49189
Iter: 53 The mean Y0 is 5.011867 Variance is: 35.197403
Iter: 54 The mean Y0 is 4.895273 Variance is: 33.90225
Iter: 55 The mean Y0 is 5.093561 Variance is: 35.33025
Iter: 56 The mean Y0 is 4.9926276 Variance is: 36.439682
Iter: 57 The mean Y0 is 5.1641426 Variance is: 36.154716
Iter: 58 The mean Y0 is 5.0523596 Variance is:

Iter: 88 The mean Y0 is 5.361599 Variance is: 41.26507
Iter: 89 The mean Y0 is 5.1392016 Variance is: 38.1638
Iter: 90 The mean Y0 is 5.037714 Variance is: 33.723194
Iter: 91 The mean Y0 is 5.130986 Variance is: 35.292034
Iter: 92 The mean Y0 is 4.947697 Variance is: 34.876038
Iter: 93 The mean Y0 is 4.715075 Variance is: 34.606243
Iter: 94 The mean Y0 is 5.2338467 Variance is: 38.803097
Iter: 95 The mean Y0 is 4.9276857 Variance is: 31.890112
Iter: 96 The mean Y0 is 5.23846 Variance is: 36.16378
Iter: 97 The mean Y0 is 4.9120564 Variance is: 35.04057
Iter: 98 The mean Y0 is 4.9077344 Variance is: 35.313026
Iter: 99 The mean Y0 is 5.0951166 Variance is: 35.94328
Epoch: 6
Iter: 0 The mean Y0 is 5.1572347 Variance is: 38.590675
Iter: 1 The mean Y0 is 4.976633 Variance is: 38.912228
Iter: 2 The mean Y0 is 5.28595 Variance is: 38.038193
Iter: 3 The mean Y0 is 5.229484 Variance is: 40.223087
Iter: 4 The mean Y0 is 4.9957504 Variance is: 35.438435
Iter: 5 The mean Y0 is 5.370889 Variance is:

...

Iter: 9 The mean Y0 is 5.4346676 Variance is: 42.79499
Iter: 10 The mean Y0 is 4.8658543 Variance is: 37.152744
Iter: 11 The mean Y0 is 5.2653623 Variance is: 39.35236
Iter: 12 The mean Y0 is 5.5048275 Variance is: 35.58831
Iter: 13 The mean Y0 is 4.9533796 Variance is: 36.712254
Iter: 14 The mean Y0 is 5.4002333 Variance is: 37.830585
Iter: 15 The mean Y0 is 5.571705 Variance is: 41.13377
Iter: 16 The mean Y0 is 5.214217 Variance is: 37.13207
Iter: 17 The mean Y0 is 5.283515 Variance is: 40.23543
Iter: 18 The mean Y0 is 5.187683 Variance is: 36.28286
Iter: 19 The mean Y0 is 5.314037 Variance is: 38.144012
Iter: 20 The mean Y0 is 4.985639 Variance is: 34.00065
Iter: 21 The mean Y0 is 5.1529675 Variance is: 38.395855
Iter: 22 The mean Y0 is 5.211709 Variance is: 38.123585
Iter: 23 The mean Y0 is 4.991942 Variance is: 35.44473
Iter: 24 The mean Y0 is 5.189191 Variance is: 39.116024
Iter: 25 The mean Y0 is 4.8762007 Variance is: 32.036762
Iter: 26 The mean Y0 is 5.2737036 Variance is: 35.

Iter: 56 The mean Y0 is 5.102734 Variance is: 34.27965
Iter: 57 The mean Y0 is 5.0792084 Variance is: 35.655018
Iter: 58 The mean Y0 is 5.0626464 Variance is: 36.70649
Iter: 59 The mean Y0 is 4.808985 Variance is: 30.751925
Iter: 60 The mean Y0 is 5.062127 Variance is: 36.911884
Iter: 61 The mean Y0 is 4.909317 Variance is: 36.37458
Iter: 62 The mean Y0 is 5.0937786 Variance is: 36.346542
Iter: 63 The mean Y0 is 5.067948 Variance is: 34.15393
Iter: 64 The mean Y0 is 5.045536 Variance is: 34.810726
Iter: 65 The mean Y0 is 5.0510325 Variance is: 34.70504
Iter: 66 The mean Y0 is 5.014136 Variance is: 37.116238
Iter: 67 The mean Y0 is 4.6917677 Variance is: 32.734932
Iter: 68 The mean Y0 is 5.0223894 Variance is: 35.81823
Iter: 69 The mean Y0 is 4.807309 Variance is: 34.47334
Iter: 70 The mean Y0 is 4.9881597 Variance is: 35.521126
Iter: 71 The mean Y0 is 4.967171 Variance is: 35.22936
Iter: 72 The mean Y0 is 5.3109856 Variance is: 37.469555
Iter: 73 The mean Y0 is 5.071206 Variance is: 35

Iter: 3 The mean Y0 is 5.367762 Variance is: 39.592583
Iter: 4 The mean Y0 is 4.888237 Variance is: 35.558533
Iter: 5 The mean Y0 is 5.392702 Variance is: 38.451286
Iter: 6 The mean Y0 is 4.932451 Variance is: 33.931892
Iter: 7 The mean Y0 is 5.0527616 Variance is: 35.76637
Iter: 8 The mean Y0 is 5.133842 Variance is: 37.11065
Iter: 9 The mean Y0 is 5.3257174 Variance is: 39.419113
Iter: 10 The mean Y0 is 5.049692 Variance is: 35.73805
Iter: 11 The mean Y0 is 4.823501 Variance is: 35.22838
Iter: 12 The mean Y0 is 5.1510267 Variance is: 38.788177
Iter: 13 The mean Y0 is 5.122097 Variance is: 36.671555
Iter: 14 The mean Y0 is 5.397993 Variance is: 41.393127
Iter: 15 The mean Y0 is 5.12265 Variance is: 37.39768
Iter: 16 The mean Y0 is 4.970603 Variance is: 35.175476
Iter: 17 The mean Y0 is 5.324105 Variance is: 36.04901
Iter: 18 The mean Y0 is 5.216621 Variance is: 35.690224
Iter: 19 The mean Y0 is 5.4321723 Variance is: 41.08154
Iter: 20 The mean Y0 is 4.8746285 Variance is: 35.618294
It

Iter: 50 The mean Y0 is 4.6659403 Variance is: 32.372925
Iter: 51 The mean Y0 is 5.025596 Variance is: 35.63418
Iter: 52 The mean Y0 is 5.235611 Variance is: 41.59674
Iter: 53 The mean Y0 is 4.9965615 Variance is: 36.16534
Iter: 54 The mean Y0 is 4.8306265 Variance is: 36.291164
Iter: 55 The mean Y0 is 5.330405 Variance is: 37.710102
Iter: 56 The mean Y0 is 5.183987 Variance is: 36.917416
Iter: 57 The mean Y0 is 4.9206305 Variance is: 32.504593
Iter: 58 The mean Y0 is 5.115601 Variance is: 36.273502
Iter: 59 The mean Y0 is 5.011812 Variance is: 33.783466
Iter: 60 The mean Y0 is 4.991125 Variance is: 34.767887
Iter: 61 The mean Y0 is 4.8697724 Variance is: 34.786163
Iter: 62 The mean Y0 is 4.999278 Variance is: 33.68893
Iter: 63 The mean Y0 is 4.9458394 Variance is: 35.631004
Iter: 64 The mean Y0 is 5.1950903 Variance is: 37.02429
Iter: 65 The mean Y0 is 4.9699764 Variance is: 37.400436
Iter: 66 The mean Y0 is 5.0796866 Variance is: 37.401295
Iter: 67 The mean Y0 is 4.9430976 Variance i

Iter: 97 The mean Y0 is 5.217831 Variance is: 35.764824
Iter: 98 The mean Y0 is 5.319771 Variance is: 36.484295
Iter: 99 The mean Y0 is 5.108325 Variance is: 33.81962
Epoch: 50


In [50]:
import os

In [51]:
files_dir=os.getcwd()+'\\eg2_trained_dataDim1\\'
file_names=os.listdir(files_dir)
pred_vec=[]

In [52]:
for file in file_names:
    df=pd.read_csv(files_dir+file)
    df_temp=df[41:]
    lw=df_temp.loss_var.mean()-2.0*df_temp.loss_var.std()
    up=df_temp.loss_var.mean()+2.0*df_temp.loss_var.std()
    df1=df_temp[df_temp.loss_var>=lw]
    df1=df1[df1.loss_var<=up]
    
    pred_vec.append(df1.y_pred.mean())

In [54]:
est_vec=np.array(pred_vec)

In [57]:
est_vec.mean()

5.038016669637957

In [55]:
2*est_vec.std()+ est_vec.mean(),-2*est_vec.std()+ est_vec.mean()

(5.104465306817219, 4.971568032458696)